## Data Collection using API
In this notebook, I will perform data preprocessing to prepare the dataset for further analysis and modeling. Data preprocessing is a crucial step in any Data Science project as it ensures the dataset is clean, structured, and suitable for building reliable models. This notebook will cover the following key tasks:

- Data Cleaning: Identifying and handling missing values, duplicates, and outliers to improve data quality.
- Feature Engineering: Creating new features and transforming existing ones to enhance the predictive power of the dataset.
- Data Transformation: Encoding categorical variables, scaling numerical features, and dealing with date fields to make the data ready for analysis.
- Data Validation: Ensuring the cleaned dataset is well-structured and consistent.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import sys
!{sys.executable} -m pip install seaborn
import seaborn as sns

# Set plot style
sns.set(style="whitegrid")

In [ ]:
import requests
import pandas as pd

# Set up base URL and indicators
base_url = "http://api.worldbank.org/v2/country/all/indicator/"
indicators = [
    "NY.GDP.MKTP.CD", "NY.GDP.PCAP.CD", "FP.CPI.TOTL",
    "BX.KLT.DINV.CD.WD", "NE.EXP.GNFS.ZS", "SP.POP.TOTL",
    "SP.POP.GROW", "SP.URB.TOTL.IN.ZS", "SP.DYN.LE00.IN",
    "SL.UEM.TOTL.ZS", "SH.DYN.MORT", "SH.XPD.CHEX.GD.ZS",
    "SH.STA.MALN.ZS", "SE.ADT.LITR.ZS", "SE.PRM.ENRR",
    "EN.ATM.CO2E.PC", "EG.ELC.ACCS.ZS", "EG.FEC.RNEW.ZS"
]

# Initialize an empty DataFrame to store results
df_combined = pd.DataFrame()

# Specify the date range for data collection
date_range = "1980:2024"

# Loop over each indicator to fetch data
for indicator in indicators:
    url = f"{base_url}{indicator}?format=json&date={date_range}&per_page=10000"
    response = requests.get(url)
    data = response.json()

    # Check if data contains actual records (not metadata)
    if len(data) > 1 and isinstance(data[1], list):
        indicator_data = data[1]

        # Extract records into a list of dictionaries
        records = []
        for entry in indicator_data:
            # Ensure each entry contains all required keys
            record = {
                'country': entry['country']['value'] if 'country' in entry else None,
                'countryiso3code': entry.get('countryiso3code', None),
                'date': entry.get('date', None),
                indicator: entry.get('value', None)  # Use indicator as column name for 'value'
            }
            records.append(record)

        # Create a DataFrame for this indicator
        df = pd.DataFrame(records)

        # Merge this indicator's data with the combined DataFrame
        if df_combined.empty:
            df_combined = df
        else:
            df_combined = pd.merge(df_combined, df, on=['country', 'countryiso3code', 'date'], how='outer')

# Save the combined data to a CSV file
df_combined.to_csv("world_bank_combined_data.csv", index=False)
print("Data saved to 'world_bank_combined_data.csv'")


Data saved to 'world_bank_combined_data.csv'


## Data Processing and Cleaning

In [ ]:
# Drop columns with more than 50% missing values
threshold = len(df) * 0.5
df_cleaned = df_combined.dropna(thresh=threshold, axis=1)

# Impute missing values with the median for columns with moderate missing data
for column in df_cleaned.select_dtypes(include=['float64']).columns:
    df_cleaned[column].fillna(df_cleaned[column].median(), inplace=True)

# Rename columns for readability
df_cleaned.rename(columns={
    'NY.GDP.MKTP.CD': 'GDP (USD)',
    'NY.GDP.PCAP.CD': 'GDP per Capita (USD)',
    'FP.CPI.TOTL': 'Inflation Rate (%)',
    'BX.KLT.DINV.CD.WD': 'FDI (USD)',
    'NE.EXP.GNFS.ZS': 'Exports (% of GDP)',
    'SP.POP.TOTL': 'Population Total',
    'SP.POP.GROW': 'Population Growth (%)',
    'SP.URB.TOTL.IN.ZS': 'Urban Population (%)',
    'SP.DYN.LE00.IN': 'Life Expectancy (Years)',
    'SL.UEM.TOTL.ZS': 'Unemployment Rate (%)',
    'SH.DYN.MORT': 'Child Mortality Rate',
    'SH.XPD.CHEX.GD.ZS': 'Health Expenditure (% of GDP)',
    'SH.STA.MALN.ZS': 'Underweight Prevalence (% of children)',
    'SE.ADT.LITR.ZS': 'Literacy Rate (%)',
    'SE.PRM.ENRR': 'Primary School Enrollment (%)',
    'EG.ELC.ACCS.ZS': 'Access to Electricity (%)',
    'EG.FEC.RNEW.ZS': 'Renewable Energy Consumption (%)'
}, inplace=True)

# Convert `date` column to datetime (keeping it as year)
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'], format='%Y')

# Display information after cleaning and a sample of the cleaned data
df_cleaned.head()

<ipython-input-68-1e31108535cf>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[column].fillna(df_cleaned[column].median(), inplace=True)
<ipython-input-68-1e31108535cf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[column].fillna(df_cleaned[column].median(), inplace=True)
<ipython-input-68-1e31108535cf>:10: SettingWi

,country,countryiso3code,date,GDP (USD),GDP per Capita (USD),Inflation Rate (%),FDI (USD),Exports (% of GDP),Population Total,Population Growth (%),Urban Population (%),Life Expectancy (Years),Unemployment Rate (%),Child Mortality Rate,Primary School Enrollment (%),Access to Electricity (%),Renewable Energy Consumption (%)
0,Afghanistan,AFG,1980-01-01,2.771514e+10,3247.146522,81.1434,9000000.0,29.426465,12486631.0,-3.924172,15.995,39.618,6.260741,246.0,46.066292,98.2,22.8
1,Afghanistan,AFG,1981-01-01,2.771514e+10,3247.146522,81.1434,180000.0,29.426465,11155195.0,-11.275324,16.562,40.164,6.260741,239.7,52.377140,98.2,22.8
2,Afghanistan,AFG,1982-01-01,2.771514e+10,3247.146522,81.1434,100000.0,29.426465,10088289.0,-10.053006,17.147,37.766,6.260741,233.2,21.814550,98.2,22.8
3,Afghanistan,AFG,1983-01-01,2.771514e+10,3247.146522,81.1434,0.0,29.426465,9951449.0,-1.365708,17.747,38.187,6.260741,226.7,100.795704,98.2,22.8
4,Afghanistan,AFG,1984-01-01,2.771514e+10,3247.146522,81.1434,0.0,29.426465,10243686.0,2.894335,18.365,33.329,6.260741,220.3,27.096519,98.2,22.8


## Feature Engineering

In [ ]:
# Copy the DataFrame to avoid modifying the original with new features
df_feature_engineered = df_cleaned.copy()

# Feature 1: GDP Growth Rate - Calculate year-over-year growth rate in GDP
df_feature_engineered['GDP Growth Rate (%)'] = df_feature_engineered.groupby('country')['GDP (USD)'].pct_change() * 100

# Feature 2: Urbanization Rate Change - Calculate the year-over-year change in Urban Population (%)
df_feature_engineered['Urbanization Rate Change (%)'] = df_feature_engineered.groupby('country')['Urban Population (%)'].diff()

# Feature 3: Unemployment Growth Rate - Calculate the year-over-year change in Unemployment Rate (%)
df_feature_engineered['Unemployment Growth Rate (%)'] = df_feature_engineered.groupby('country')['Unemployment Rate (%)'].diff()

# Feature 4: Country Grouping by Regions using 'country' column for a simplified mapping
# Define a basic region mapping based on common country names (example subset)
region_mapping_simplified = {
    'Afghanistan': 'Asia', 'United States': 'North America', 'Germany': 'Europe',
    'India': 'Asia', 'Brazil': 'South America', 'South Africa': 'Africa',
    'Canada': 'North America', 'United Kingdom': 'Europe', 'China': 'Asia', 'Japan': 'Asia'
}
# Apply the mapping to create a new column 'Region' based on 'country'
df_feature_engineered['Region'] = df_feature_engineered['country'].map(region_mapping_simplified)

# Step 5: Binary Flag for High vs. Low GDP
# Define the median GDP value as the threshold for high vs low GDP, using the standardized GDP value
median_gdp = df_feature_engineered['GDP (USD)'].median()
df_feature_engineered['High GDP'] = (df_feature_engineered['GDP (USD)'] >= median_gdp).astype(int)

# Display the new features and some sample data
df_feature_engineered.head()

,country,countryiso3code,date,GDP (USD),GDP per Capita (USD),Inflation Rate (%),FDI (USD),Exports (% of GDP),Population Total,Population Growth (%),...,Unemployment Rate (%),Child Mortality Rate,Primary School Enrollment (%),Access to Electricity (%),Renewable Energy Consumption (%),GDP Growth Rate (%),Urbanization Rate Change (%),Unemployment Growth Rate (%),Region,High GDP
0,Afghanistan,AFG,1980-01-01,2.771514e+10,3247.146522,81.1434,9000000.0,29.426465,12486631.0,-3.924172,...,6.260741,246.0,46.066292,98.2,22.8,NaN,NaN,NaN,Asia,1
1,Afghanistan,AFG,1981-01-01,2.771514e+10,3247.146522,81.1434,180000.0,29.426465,11155195.0,-11.275324,...,6.260741,239.7,52.377140,98.2,22.8,0.0,0.567,0.0,Asia,1
2,Afghanistan,AFG,1982-01-01,2.771514e+10,3247.146522,81.1434,100000.0,29.426465,10088289.0,-10.053006,...,6.260741,233.2,21.814550,98.2,22.8,0.0,0.585,0.0,Asia,1
3,Afghanistan,AFG,1983-01-01,2.771514e+10,3247.146522,81.1434,0.0,29.426465,9951449.0,-1.365708,...,6.260741,226.7,100.795704,98.2,22.8,0.0,0.600,0.0,Asia,1
4,Afghanistan,AFG,1984-01-01,2.771514e+10,3247.146522,81.1434,0.0,29.426465,10243686.0,2.894335,...,6.260741,220.3,27.096519,98.2,22.8,0.0,0.618,0.0,Asia,1


## Outlier Detection & Treatment

In [ ]:
# Reload the preprocessed DataFrame to reset outlier treatment
df_outlier_treated = df_feature_engineered.copy()

# Select numerical columns for IQR-based outlier detection
numeric_columns = df_outlier_treated.select_dtypes(include=['float64']).columns

# Apply the IQR method to detect and cap outliers for each numeric column
for column in numeric_columns:
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df_outlier_treated[column].quantile(0.25)
    Q3 = df_outlier_treated[column].quantile(0.75)
    IQR = Q3 - Q1  # Calculate the IQR

    # Define the outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Cap the outliers using the IQR bounds
    df_outlier_treated[column] = np.where(df_outlier_treated[column] < lower_bound, lower_bound,
                                      np.where(df_outlier_treated[column] > upper_bound, upper_bound,
                                               df_outlier_treated[column]))

# Display a summary of the adjusted DataFrame after IQR-based outlier treatment
df_outlier_treated.head()

,country,countryiso3code,date,GDP (USD),GDP per Capita (USD),Inflation Rate (%),FDI (USD),Exports (% of GDP),Population Total,Population Growth (%),...,Unemployment Rate (%),Child Mortality Rate,Primary School Enrollment (%),Access to Electricity (%),Renewable Energy Consumption (%),GDP Growth Rate (%),Urbanization Rate Change (%),Unemployment Growth Rate (%),Region,High GDP
0,Afghanistan,AFG,1980-01-01,2.771514e+10,3247.146522,81.1434,9000000.0,29.426465,12486631.0,-2.237772,...,6.260741,151.518008,86.725454,98.2,22.8,NaN,NaN,NaN,Asia,1
1,Afghanistan,AFG,1981-01-01,2.771514e+10,3247.146522,81.1434,180000.0,29.426465,11155195.0,-2.237772,...,6.260741,151.518008,86.725454,98.2,22.8,0.0,0.567,0.0,Asia,1
2,Afghanistan,AFG,1982-01-01,2.771514e+10,3247.146522,81.1434,100000.0,29.426465,10088289.0,-2.237772,...,6.260741,151.518008,86.725454,98.2,22.8,0.0,0.585,0.0,Asia,1
3,Afghanistan,AFG,1983-01-01,2.771514e+10,3247.146522,81.1434,0.0,29.426465,9951449.0,-1.365708,...,6.260741,151.518008,100.795704,98.2,22.8,0.0,0.600,0.0,Asia,1
4,Afghanistan,AFG,1984-01-01,2.771514e+10,3247.146522,81.1434,0.0,29.426465,10243686.0,2.894335,...,6.260741,151.518008,86.725454,98.2,22.8,0.0,0.618,0.0,Asia,1


In [ ]:
# Save the cleaned data to a CSV file
df_outlier_treated.to_csv("world_bank_cleaned_data.csv", index=False)
print("Data saved to 'world_bank_cleaned_data.csv'")

Data saved to 'world_bank_cleaned_data.csv'


## Scaling Data for ML

In [ ]:
from sklearn.preprocessing import StandardScaler

# Copy the cleaned dataset to avoid modifying the original
df_scaled = df_outlier_treated.copy()

# Scaling Numerical Features
# Select only the numerical columns for scaling (excluding date, country, and countryiso3code)
numerical_columns = df_scaled.select_dtypes(include=['float64']).columns

# Initialize the scaler and apply it to the numerical columns
scaler = StandardScaler()
df_scaled[numerical_columns] = scaler.fit_transform(df_scaled[numerical_columns])
df_scaled.head()

,country,countryiso3code,date,GDP (USD),GDP per Capita (USD),Inflation Rate (%),FDI (USD),Exports (% of GDP),Population Total,Population Growth (%),...,Unemployment Rate (%),Child Mortality Rate,Primary School Enrollment (%),Access to Electricity (%),Renewable Energy Consumption (%),GDP Growth Rate (%),Urbanization Rate Change (%),Unemployment Growth Rate (%),Region,High GDP
0,Afghanistan,AFG,1980-01-01,-0.584336,-0.469566,-0.001074,-0.671428,-0.228354,-0.514680,-2.851975,...,-0.125412,2.300926,-1.757191,0.506295,-0.232847,NaN,NaN,NaN,Asia,1
1,Afghanistan,AFG,1981-01-01,-0.584336,-0.469566,-0.001074,-0.673307,-0.228354,-0.532731,-2.851975,...,-0.125412,2.300926,-1.757191,0.506295,-0.232847,-0.531087,0.707289,0.082682,Asia,1
2,Afghanistan,AFG,1982-01-01,-0.584336,-0.469566,-0.001074,-0.673324,-0.228354,-0.547195,-2.851975,...,-0.125412,2.300926,-1.757191,0.506295,-0.232847,-0.531087,0.762381,0.082682,Asia,1
3,Afghanistan,AFG,1983-01-01,-0.584336,-0.469566,-0.001074,-0.673346,-0.228354,-0.549051,-2.189152,...,-0.125412,2.300926,0.048065,0.506295,-0.232847,-0.531087,0.808291,0.082682,Asia,1
4,Afghanistan,AFG,1984-01-01,-0.584336,-0.469566,-0.001074,-0.673346,-0.228354,-0.545089,1.048747,...,-0.125412,2.300926,-1.757191,0.506295,-0.232847,-0.531087,0.863383,0.082682,Asia,1


In [ ]:
# Save the scaled data to a CSV file
df_scaled.to_csv("world_bank_scaled_data.csv", index=False)
print("Data saved to 'world_bank_scaled_data.csv'")

Data saved to 'world_bank_scaled_data.csv'
